In [3]:
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import os
import openai
import json

import requests
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored


load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

if os.getenv("OPENAI_API_KEY") is not None:
    print ("OPENAI_API_KEY is ready")
else:
    print ("OPENAI_API_KEY environment variable not found")

GPT_MODEL = "gpt-3.5-turbo-0613"

OPENAI_API_KEY is ready


In [4]:
# Read data about the product

with open('/Users/vladbordei/Documents/Development/ProductExplorer/data/interim/short_product_information.json') as file:
    json_string = file.read()
    general_product_data = json.loads(json_string)

In [5]:
#asin_list_path = './data/external/asin_list.csv'
asin_list_path = '/Users/vladbordei/Documents/Development/ProductExplorer/data/external/asin_list.csv'
asin_list = pd.read_csv(asin_list_path)['asin'].tolist()

In [10]:
attribute_clusters_with_percentage_path = '/Users/vladbordei/Documents/Development/ProductExplorer/data/interim/attribute_clusters_with_percentage.csv'
attributes = pd.read_csv(attribute_clusters_with_percentage_path)

In [12]:
attributes.drop(columns=['Unnamed: 0'], inplace=True)

In [15]:
set(attributes['Attribute'])

{'Buyer Motivation',
 'Customer Expectations',
 'Danger Appraisal',
 'Design and Appearance',
 'Durability',
 'Ease of Use',
 'How the product is used',
 'Parts and Components',
 'Price',
 'Quality',
 'Season',
 'Setup and Instructions',
 'Size and Fit',
 'User Description',
 'When the product is used',
 'Where the product is used'}

In [18]:
attributes[attributes['Attribute'] ==  'Customer Expectations']

,Attribute,cluster_label,positive_sentiment,negative_sentiment,Rating,id,asin,URL,observation_count,rating_avg,attribute_percentage
7,Customer Expectations,Durable and easy-to-use raised bed,0.989524,0.001497,[5],[6],['B09D7QDLYC'],['https://www.amazon.com/gp/customer-reviews/R...,1,5,14.285714
8,Customer Expectations,Durable bed,0.774938,0.053037,[3],[3],['B0BG2HP1PJ'],['https://www.amazon.com/gp/customer-reviews/R...,1,3,14.285714
9,Customer Expectations,Easy assembly,0.003215,0.937854,[2],[9],['B0BHS6YL2F'],['https://www.amazon.com/gp/customer-reviews/R...,1,2,14.285714
10,Customer Expectations,Good size and sturdy product,0.986625,0.001518,[5],[2],['B09K3557Q8'],['https://www.amazon.com/gp/customer-reviews/R...,1,5,14.285714
11,Customer Expectations,Plant growth expectations,0.994206,0.001794,[5],[10],['B0BNPLKNJB'],['https://www.amazon.com/gp/customer-reviews/R...,1,5,14.285714
12,Customer Expectations,Sturdy and functional gardening space,0.943908,0.004142,[5],[8],['B0BM5ZJWQP'],['https://www.amazon.com/gp/customer-reviews/R...,1,5,14.285714
13,Customer Expectations,To help tremendously with gardening,0.993381,0.002647,[5],[5],['B0BGND2M55'],['https://www.amazon.com/gp/customer-reviews/R...,1,5,14.285714
